# LSTM Based Language Model - Part II
A language model looks at the context to generate next set of words. This context is also called as a sliding window which moves across the input sentence from left to right(right to left for language which are written from right to left). 

This is the second notebook with same layout. We present two variants of the model, one with stacked LSTM layers and one with a bidirectional LSTM layer.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PacktPublishing/Hands-On-Generative-AI-with-Python-and-TensorFlow-2/blob/master/Chapter_9/language_model_stacked_lstm.ipynb)

## Import Required Libraries

In [1]:
import os
import math
import numpy as np
import tensorflow as tf

In [2]:
print("Tensorflow version={}".format(tf.__version__))

Tensorflow version=2.3.0


## Load Dataset

In [3]:
# https://www.gutenberg.org/ebooks/2600
datafile_path = r'warpeace_2600-0.txt'

In [4]:
# Load the text file
text = open(datafile_path, 'rb').read().decode(encoding='utf-8')
print ('Book contains a total of {} characters'.format(len(text)))

Book contains a total of 2051344 characters


In [5]:
idx = 8091
print(text[idx:idx+500])


BOOK ONE: 1805





CHAPTER I

“Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don’t tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by that
Antichrist—I really believe he is Antichrist—I will have nothing
more to do with you and you are no longer my friend, no longer my
‘faithful slave,’ as you call yourself! But how do you do? I see I
have frightened you—sit down and tell me al


In [6]:
# We remove first 8k characters to remove 
# details related to project gutenberg
text = text [8091:]

In [7]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

100 unique characters


## Prepare Dataset
+ Dictionary of character to index mapping
+ Inverse mapping of index to character mapping

In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '(' :   4,
  ')' :   5,
  '*' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '0' :  10,
  '1' :  11,
  '2' :  12,
  '3' :  13,
  '4' :  14,
  '5' :  15,
  '6' :  16,
  '7' :  17,
  '8' :  18,
  '9' :  19,
  ...
}


### Sample Output

In [10]:
print ('{} ---- char-2-int ----  {}'.format(repr(text[40:60]), text_as_int[40:60]))

'\n“Well, Prince, so G' ---- char-2-int ----  [ 0 98 45 53 60 60  7  2 38 66 57 62 51 53  7  2 67 63  2 29]


### Prepare Batch of Training Samples
+ Sequence length limit to 100
+ Use ``tf.data`` API to prepare batches

In [11]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2char[i.numpy()])




B
O
O
K
 
O
N
E


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(10):
    print(repr(''.join(idx2char[item.numpy()])))
    print("-"*110)

'\r\nBOOK ONE: 1805\r\n\r\n\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\n“Well, Prince, so Genoa and Lucca are now just family estate'
--------------------------------------------------------------------------------------------------------------
's of the\r\nBuonapartes. But I warn you, if you don’t tell me that this means war,\r\nif you still try to'
--------------------------------------------------------------------------------------------------------------
' defend the infamies and horrors perpetrated by that\r\nAntichrist—I really believe he is Antichrist—I '
--------------------------------------------------------------------------------------------------------------
'will have nothing\r\nmore to do with you and you are no longer my friend, no longer my\r\n‘faithful slave'
--------------------------------------------------------------------------------------------------------------
',’ as you call yourself! But how do you do? I see I\r\nhave frightened you—sit down and tell me all the'
------

### Prepare Input->Target samples

In [13]:
def split_input_target(chunk):
    """
    Utility which takes a chunk of input text and target as one position shifted form of input chunk.
    Parameters:
        chunk: input list of words
    Returns:
        Tuple-> input_text(i.e. chunk minus last word),target_text(input chunk minus the first word)
    """
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\r\nBOOK ONE: 1805\r\n\r\n\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\n“Well, Prince, so Genoa and Lucca are now just family estat'
Target data: '\nBOOK ONE: 1805\r\n\r\n\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\n“Well, Prince, so Genoa and Lucca are now just family estate'


In [15]:
# Batch size
BATCH_SIZE = 128
# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

In [16]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print("Dataset Shape={}".format(dataset))

Dataset Shape=<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>


## Prepare Language Model

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size,is_bidirectional=False):
    """
    Utility to create a model object.
    Parameters:
        vocab_size: number of unique characters
        embedding_dim: size of embedding vector. This typically in powers of 2, i.e. 64, 128, 256 and so on
        rnn_units: number of LSTM units to be used
        batch_size: batch size for training the model
    Returns:
        tf.keras model object
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]))
    if is_bidirectional:
      model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')))
    else:
      model.add(tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

### Define the Model Parameters

In [18]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [19]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          25600     
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         5246976   
_________________________________________________________________
lstm_1 (LSTM)                (128, None, 1024)         8392704   
_________________________________________________________________
dense (Dense)                (128, None, 100)          102500    
Total params: 13,767,780
Trainable params: 13,767,780
Non-trainable params: 0
_________________________________________________________________


In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [22]:
model.compile(optimizer='adam', loss=loss)

### Setup Callbacks

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = r'data/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
EPOCHS = 25
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25
158/158 [==============================] - 30s 188ms/step - loss: 2.6990
Epoch 2/25
158/158 [==============================] - 30s 188ms/step - loss: 1.7935
Epoch 3/25
158/158 [==============================] - 30s 188ms/step - loss: 1.4524
Epoch 4/25
158/158 [==============================] - 30s 188ms/step - loss: 1.3151
Epoch 5/25
158/158 [==============================] - 30s 188ms/step - loss: 1.2415
Epoch 6/25
158/158 [==============================] - 30s 187ms/step - loss: 1.1903
Epoch 7/25
158/158 [==============================] - 30s 188ms/step - loss: 1.1502
Epoch 8/25
158/158 [==============================] - 30s 187ms/step - loss: 1.1153
Epoch 9/25
158/158 [==============================] - 30s 188ms/step - loss: 1.0830
Epoch 10/25
158/158 [==============================] - 30s 188ms/step - loss: 1.0520
Epoch 11/25
158/158 [==============================] - 30s 188ms/step - loss: 1.0195
Epoch 12/25
158/158 [==============================] - 30s 189ms/step - lo

## Generate Fake Text

### Load Latest Checkpoint

In [25]:
# fetch the latest checkpoint from the model directory
tf.train.latest_checkpoint(checkpoint_dir)

'data/training_checkpoints/ckpt_25'

In [26]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            25600     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 100)            102500    
Total params: 13,767,780
Trainable params: 13,767,780
Non-trainable params: 0
_________________________________________________________________


### Utility Function to Generate Text

In [28]:
def generate_text(model, mode='greedy', context_string='Hello', num_generate=1000, 
                  temperature=1.0):
    """
    Utility to generate text given a trained model and context
    Parameters:
        model: tf.keras object trained on a sufficiently sized corpus
        mode: decoding mode. Default is greedy. Other mode is
              sampling (set temperature)
        context_string: input string which acts as context for the model
        num_generate: number of characters to be generated
        temperature: parameter to control randomness of outputs
    Returns:
        string : context_string+text_generated
    """

    # vectorizing: convert context string into string indices
    input_eval = [char2idx[s] for s in context_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # String for generated characters
    text_generated = []
    beam_input_predictions = []
    model.reset_states()
    # Loop till required number of characters are generated
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        if mode == 'greedy':
          predicted_id = np.argmax(predictions[0])
          
        elif mode == 'sampling':
          # temperature helps control the character returned by the model.
          predictions = predictions / temperature
          # Sampling over a categorical distribution
          predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # predicted character acts as input for next step
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (context_string + ''.join(text_generated))

### Greedy Decoding

In [29]:
print(generate_text(model, mode= 'greedy', context_string=u"It was in July, 1805",num_generate=50))

It was in July, 1805” and the general course of which she did not
und


### Sampled Decoding

In [30]:
print(generate_text(model, mode= 'sampling', context_string=u"It was in July, 1805",num_generate=100,temperature=0.3))

It was in July, 1805, and the general
who attracted him as a child, her own son, who was still five him a broad and in 


In [31]:
print(generate_text(model, mode= 'sampling', context_string=u"It was in July, 1805",num_generate=100,temperature=0.6))

It was in July, 1805, and two old
prince and he and went out into the deserted field of reconciling the trousseau is at


In [32]:
print(generate_text(model, mode= 'sampling', context_string=u"It was in July, 1805",num_generate=100,temperature=0.9))

It was in July, 1805, and heaven only would the
count never died so easily to do, since Boguchárovo, ready to
full spe
